In [1]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs               
import cartopy.feature as cfeature         
from cartopy.util import add_cyclic_point  
import xarray as xr                        
import numpy as np 

In [2]:
ozone_dataset = xr.open_dataset("/glade/campaign/acom/acom-climate/UTLS/shawnh/archive/FCnudged_f09.mam.mar27.2000_2021.002/atm/proc/tseries/month_1/FCnudged_f09.mam.mar27.2000_2021.002.cam.h0.O3.200201-202412.nc")
pdeldry_dataset = xr.open_dataset("/glade/campaign/acom/acom-climate/UTLS/shawnh/archive/FCnudged_f09.mam.mar27.2000_2021.002/atm/proc/tseries/month_1/FCnudged_f09.mam.mar27.2000_2021.002.cam.h0.PDELDRY.200201-202412.nc")
ps_dataset = xr.open_dataset('/glade/campaign/acom/acom-climate/UTLS/shawnh/archive/FCnudged_f09.mam.mar27.2000_2021.002/atm/proc/tseries/month_1/FCnudged_f09.mam.mar27.2000_2021.002.cam.h0.PS.200201-202412.nc')

In [3]:
ozone = ozone_dataset["O3"]
# ozone_ppb = ozone.copy() * 10**9 convert to ppb, copy for no extra calculation

lat = ozone.coords['lat']
lon = ozone.coords['lon']

Calculating pressure at every hybrid level

p(k) = a(k) * p0 + b(k) * ps

In [4]:
p0 = ozone_dataset["P0"]
hyai = ozone_dataset["hyai"]
hybi = ozone_dataset["hybi"]
ps = ps_dataset['PS'].isel(time=0)
lev = ozone_dataset.coords['lev']
num_lev = lev.shape[0]
pdeldry = pdeldry_dataset['PDELDRY']
pdeldry = pdeldry.copy() / 100


In [5]:
# constants / conversion factor
NAv = 6.0221415e+23                       # molecules in mole
g = 9.81                                  # gravity
MWair = 28.94                             # g/mol
xp_const = (NAv * 10)/(MWair*g)           # scaling factor, pa to hPa and cm to m
DU_CONVERSION = 2.69 * 10**16

In [62]:
# Initialize pressure edge arrays
mod_press_low = xr.zeros_like(ozone)
mod_press_top = xr.zeros_like(ozone)

num_level = [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
# Calculate pressure edge arrays
# CAM-chem layer indices start at the top and end at the bottom
for i in range(num_lev):
    mod_press_top[i,:,:] = hyai[i]*p0 + hybi[i]*ps
    mod_press_low[i,:,:] = hyai[i+1]*p0 + hybi[i+1]*ps
    
# Delta P in hPa
mod_deltap = (mod_press_low - mod_press_top)/100
#print(mod_press_low[:,0,0])
#print(mod_press_top[:,0,0])
#print(mod_deltap[:,0,0])

In [68]:
ozone_column = xr.dot(mod_deltap, xp_const*ozone, dims=['lev'])

In [69]:
ozone_du_column = ozone_column.copy() / DU_CONVERSION

In [75]:
ozone_du_column[1]

# average over months from 2002-2024, find highest level interfaces (16-31)

<xarray.DataArray 'O3' (lat: 192, lon: 288)> Size: 221kB
array([[121.03169 , 121.03169 , 121.03169 , ..., 121.03169 , 121.03169 ,
        121.03169 ],
       [120.753235, 120.75584 , 120.75891 , ..., 120.74464 , 120.747696,
        120.75066 ],
       [120.65432 , 120.6585  , 120.66233 , ..., 120.63881 , 120.64461 ,
        120.649765],
       ...,
       [213.92314 , 213.86359 , 213.8053  , ..., 214.1131  , 214.04796 ,
        213.98456 ],
       [214.54851 , 214.51518 , 214.48291 , ..., 214.64989 , 214.61539 ,
        214.5818  ],
       [215.17804 , 215.17804 , 215.17804 , ..., 215.17804 , 215.17804 ,
        215.17804 ]], dtype=float32)
Coordinates:
  * lat      (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon      (lon) float64 2kB 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    time     datetime64[ns] 8B 2002-03-01